<a href="https://colab.research.google.com/github/hadagarcia/zerotogans/blob/main/notebooks/01_tensor_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1zK1Aq0O-0YZjZkcHBADX3SuaOQ2L40pO')

# PyTorch functions you need to know to work with tensors

These are some common functions I've found on several NN PyTorch examples, even on personal experience I've seen this functions or similar ones implemented on Machine Learning and Deep Learning tutorials. 

- 1. torch.tensor
- 2. torch.transpose
- 3. torch.max
- 4. torch.eq
- 5. torch.unique
- 6. Serialization, tensor.save / tensor.load

Before we begin, let's install and import PyTorch

In [ ]:
# Import torch and other required modules
import torch

## **1. torch.tensor**

A torch tensor is a multi-dimensional matrix containing elements of a single data type. Torch defines 10 tensor types, although a very used type is torch.float64

There are many input options to construct a PyTorch tensor, mainly using the advantage of the NumPy bridge, making it very easy to use these arrays.

---

Syntaxis: 
>**torch.tensor**(*data(array_like)*) : Can be a list, tuple, NumPy ndarray, scalar and other types.

Optional arguments:

* dtype (torch.dtype)
* device (torch.device)
* requires_grad (bool)
* pin_memory (bool)

---

#### 1.1 Simple NumPy array as input

Two interesting parameters are *device* and *requires_grad*, where *device* is the main difference why we are not using NumPy arrays; it offers the possibility to allocate the Tensor on 'cpu' or 'cuda' devices, 'cuda' devices are usually GPUs allowing faster computations.

*requires_grad* if True, it starts forming a backward graph that tracks every operation applied on it to calculate the gradients using something called a dynamic computation graph (DCG)

In [ ]:
import numpy as np
nArray = np.ones(5)
print(nArray)

[1. 1. 1. 1. 1.]


In [ ]:
f1Tensor1 = torch.tensor(nArray, dtype=torch.float64, requires_grad=True)
print(f1Tensor1)
print(f1Tensor1.grad) # Gradients, in this case None

tensor([1., 1., 1., 1., 1.], dtype=torch.float64, requires_grad=True)
None


#### 1.2 Constructing a tensor using data from a CSV file.

A real life example would be to get Data on CSV format, after applying some cleaning it can be used as input to construct a PyTorch Tensor as shown on this example.

In [ ]:
import pandas as pd

In [ ]:
mockdata = pd.read_csv('MOCK_DATA.csv')
sample = mockdata[['r', 'g', 'b']].sample(5) # Using a sample of 5 items, only for demonstration.
sample

,r,g,b
58,75,62,186
42,89,224,249
74,78,185,149
3,151,70,89
92,32,198,24


In [ ]:
print(sample.values)
print('Type: {} and shape: {}'.format(type(sample), sample.shape))

[[ 75  62 186]
 [ 89 224 249]
 [ 78 185 149]
 [151  70  89]
 [ 32 198  24]]
Type: <class 'pandas.core.frame.DataFrame'> and shape: (5, 3)


In [ ]:
# Our sample is a Pandas Dataframe, we need to convert it to a NumPy array
tensorRGB = torch.tensor(sample.to_numpy())
print(tensorRGB)
print('Tensor type: {}'.format(type(tensorRGB)))

tensor([[ 75,  62, 186],
        [ 89, 224, 249],
        [ 78, 185, 149],
        [151,  70,  89],
        [ 32, 198,  24]])
Tensor type: <class 'torch.Tensor'>


#### 1.3 [ Error: ValueError ] Not a valid Tensor

The input should have a compatible shape

In [ ]:
torch.tensor([[1, 2], [3, 4, 5]])

ValueError: expected sequence of length 2 at dim 1 (got 3)

## **2. torch.transpose**

Sometimes is needed to swap the tensor dimensions, in order to do that we can use this function, which returns the transposed version.

Using this function we can only transpose over 2D.

>$A_{3,2} = \begin{pmatrix} a_{1,1} & a_{1,2} \\  a_{2,1} & a_{2,2} \\ a_{3,1} & a_{3,2} \end{pmatrix}$  --> $A_{2,3} = \begin{pmatrix} a_{1,1} & a_{1,2} & a_{1,3} \\ a_{2,1} & a_{2,2} & a_{2,3} \end{pmatrix}^T$

---

Syntaxis: 
>**torch.transpose**(*input(Tensor), dim0(int), dim1(int)*)

Returns:

*   output (Tensor): Transposed version of *input* 

---


#### 2.1 Transposing a tensor of shape (2, 3)

Following the rules to transpose a matrix, we'll obtain a tensor of shape (3, 2)

In [ ]:
simpleTensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
simpleTensor

tensor([[1, 2, 3],
        [4, 5, 6]])

In [ ]:
f2Tensor1 = torch.transpose(simpleTensor, 0, 1)
f2Tensor1

tensor([[1, 4],
        [2, 5],
        [3, 6]])

#### 2.2 We'll transpose over 2D in a 3D Tensor

By especifying the *dim0* and *dim1* parameters, we can define from which to which dimensions we need to transpose.

In this case only the dimensions 1 to 2 will be transposed, passing from a shape (2, 5) to (5, 2). Dimension 0 remains the same. 

In [ ]:
# 3D Tensor (3, 2, 5)
simpleTensor2 = torch.randn(3, 2, 5)
print('Tensor 1 of shape (3, 2, 5): \n{}\n'.format(simpleTensor2))

f2Tensor2 = torch.transpose(simpleTensor2, 1, 2)
print('Tensor 2 of shape (3, 5, 2): \n{}'.format(f2Tensor2))

Tensor 1 of shape (3, 2, 5): 
tensor([[[ 0.5390, -2.4411, -0.3869, -1.2065, -1.9151],
         [-0.1402,  0.0821, -0.8802, -1.0765, -2.4328]],

        [[-2.3460,  0.5511, -0.4033, -0.1044,  2.1832],
         [ 1.0715,  1.2554, -0.4236, -1.0823, -0.3150]],

        [[-0.4011, -0.6395, -0.2994,  0.7747,  0.0376],
         [-1.4594, -1.6728, -0.3633,  0.6559, -0.2181]]])

Tensor 2 of shape (3, 5, 2): 
tensor([[[ 0.5390, -0.1402],
         [-2.4411,  0.0821],
         [-0.3869, -0.8802],
         [-1.2065, -1.0765],
         [-1.9151, -2.4328]],

        [[-2.3460,  1.0715],
         [ 0.5511,  1.2554],
         [-0.4033, -0.4236],
         [-0.1044, -1.0823],
         [ 2.1832, -0.3150]],

        [[-0.4011, -1.4594],
         [-0.6395, -1.6728],
         [-0.2994, -0.3633],
         [ 0.7747,  0.6559],
         [ 0.0376, -0.2181]]])


In [ ]:
# If we need to transpose an N-Dimension Tensor, it's possible. We just need to use the function torch.Tensor.permute
nDimTensor = torch.randn(2, 3, 5)
print('nDimTensor {}: \n{}\n'.format(nDimTensor.size(), nDimTensor))

permutedTensor = nDimTensor.permute(2, 0, 1)
print('nDimTensor permuted {}: \n{}'.format(permutedTensor.size() ,permutedTensor))

nDimTensor torch.Size([2, 3, 5]): 
tensor([[[-0.6859,  0.5925, -0.7868,  0.3200,  0.4541],
         [ 0.9836,  2.1425,  1.1974, -0.8963, -0.0996],
         [-0.1130,  0.3706,  0.1200, -2.0270,  0.1104]],

        [[-0.6609,  0.8163, -0.8878,  0.7976,  1.3538],
         [-1.4436,  1.3149,  0.2626,  0.1535, -1.2025],
         [ 1.2259, -0.5591, -1.7703,  0.9217,  1.3029]]])

nDimTensor permuted torch.Size([5, 2, 3]): 
tensor([[[-0.6859,  0.9836, -0.1130],
         [-0.6609, -1.4436,  1.2259]],

        [[ 0.5925,  2.1425,  0.3706],
         [ 0.8163,  1.3149, -0.5591]],

        [[-0.7868,  1.1974,  0.1200],
         [-0.8878,  0.2626, -1.7703]],

        [[ 0.3200, -0.8963, -2.0270],
         [ 0.7976,  0.1535,  0.9217]],

        [[ 0.4541, -0.0996,  0.1104],
         [ 1.3538, -1.2025,  1.3029]]])


#### 2.3 [ Error: TypeError ] It's required to specify the first and second dimensions

In [ ]:
torch.transpose(torch.tensor([[1, 2, 3]]))

TypeError: ignored

## **3. torch.max**

Returns the maximum value of all elements for the input tensor given.

If we use the second parameter, which is the dimension where we want to find the maximums, it returns a tuple (values, indices)

---

Syntaxis: 
>**torch.max**(*input(Tensor),  dim(int)*)

Returns:

*   output (tuple): two output tensors (max, max_indices) 

---

#### 3.1 Basic example with random values tensor

Input is a tensor of 3 dimensions with random values and we'll look for the maximum value on the first dimension.

In [ ]:
randomTensor = torch.randn(2, 3, 2)
randomTensor

tensor([[[-1.1600, -0.2537],
         [-0.3126,  1.4879],
         [-0.6872,  0.6803]],

        [[-1.4099,  0.1844],
         [ 1.9199, -0.1031],
         [ 1.6564,  0.2365]]])

In [ ]:
values, max_indices = torch.max(randomTensor, 0)
print(values)
print(max_indices)

tensor([[-1.1600,  0.1844],
        [ 1.9199,  1.4879],
        [ 1.6564,  0.6803]])
tensor([[0, 1],
        [1, 0],
        [1, 0]])


#### 3.2 In this example, we're looking for the maximum on the second dimension.

The max_indices output is a tensor with size [2], because the dimension we're evaluating is the second one.

max_indices2 shows on which index was the maximum found.

In [ ]:
randomTensor2 = torch.randn(2, 3)
print(randomTensor2)

values2, max_indices2 = torch.max(randomTensor2, 1)
print('\nMax values on each row: {}\n'.format(values2))
print('Indices where the max values were found on each row: \n  row0 = row0[{}]\n  row1 = row1[{}] \n  {}'.format(max_indices2[0], max_indices2[1], max_indices2)) 

tensor([[ 1.1281, -0.6203, -0.8532],
        [-0.8200, -0.8610,  0.6337]])

Max values on each row: tensor([1.1281, 0.6337])

Indices where the max values were found on each row: 
  row0 = row0[0]
  row1 = row1[2] 
  tensor([0, 2])


In [ ]:
print(type(max_indices2))
print(max_indices2.size())

<class 'torch.Tensor'>
torch.Size([2])


#### 3.3 [ Error: IndexError ] Wrong dimension given

In this example we're asking to get the max over the second dimension, but the given input is a vector (one dimensional tensor)

In [ ]:
torch.max(torch.randn(4), 1)

IndexError: ignored

## **4. torch.eq**

Compares two tensors element-wise. 
The second tensor should be a number or a tensor whose shape is *broadcastable* with the first one.

> *broadcastable*: Broadcasting describes how tensors with different shapes are treated during arithmetic operations. Meaning the 2 tensors should be compatible in this case to be element-wise compared.

---

Syntaxis: 
>**torch.eq**(*input(Tensor), other(Tensor)*)

Returns:

*   output (Tensor): A boolean tensor with True values where *input* is equal to *other* and False elsewhere 

---

#### 4.1 Basic example

We'll use two different tensors to compare using torch.eq, which in this case will return a tensor with booleans (3, 3).

These tensors don't have the same shape, but they are broadcast compatible. If we broadcast compT1 to shape (3, 3) it looks like this:

$\begin{pmatrix} 1. & 1. & 1. \end{pmatrix}$ --> $\begin{pmatrix} 1. & 1. & 1. \\  1. & 1. & 1. \\ 1. & 1. & 1. \end{pmatrix}$

In [ ]:
compT1 = torch.ones(1, 3)
compT2 = torch.eye(3, 3)
print(compT1)
print(compT2)

tensor([[1., 1., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


In [ ]:
compT3 = torch.eq(compT1, compT2)
print(compT3)

# Another useful function is torch.all, which returns a final boolean
torch.all(compT3)

tensor([[ True, False, False],
        [False,  True, False],
        [False, False,  True]])


tensor(False)

#### 4.2 Using our previous RGB Tensor obtained from the CSV file, we can compare using as the first input a tensor of shape (1, 3)

In [ ]:
tensorRGB.shape

torch.Size([5, 3])

In [ ]:
# Let's look for the color white rgb(255, 255, 255)
torch.eq(torch.tensor([255, 255, 255]), tensorRGB)

tensor([[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]])

#### 4.3 [ Error: RuntimeError ] Tensors with none broadcastable shapes

In [ ]:
failTensor1 = torch.randn(2, 3)
failTensor2 = torch.randn(3, 2)
print('Size first tensor: {} \nSize second tensor: {}'.format(failTensor1.shape, failTensor2.shape))

torch.eq(failTensor1, failTensor2)

Size first tensor: torch.Size([2, 3]) 
Size second tensor: torch.Size([3, 2])


RuntimeError: ignored

## **5. torch.unique** 

It returns the unique elements of the input tensor.

Sorting could be slow, so if your input tensor is already sorted, it's recommended to use **torch.unique_consecutive()** instead, which avoids the sorting.

---

Syntaxis: 
>**torch.unique**(*input(Tensor), sorted(bool), return_inverse(bool), return_counts(bool), dim(int)*)

Returns:

*   output (Tensor): The output list of unique scalar elements
*   inverse_indices (Tensor): this is optional
*   counts (Tensor): this is also optional

---


#### 5.1 Basic example

First we'll use as input a tensor of shape (3, 3) with some repeated values, where the output will be a tensor with all unique values.

In [ ]:
f5Tensor = torch.tensor([[1, 3, 2], [1, 0, 6], [0, 3, 0]])
f5Tensor.shape

torch.Size([3, 3])

In [ ]:
uTensor = torch.unique(f5Tensor)
print('All unique elements: {}'.format(uTensor))
uTensor.shape

# We didn't specify over which dimension to look for uniques, so it went through all elements

All unique elements: tensor([0, 1, 2, 3, 6])


torch.Size([5])

#### 5.2 Using our previous RGB Tensor, we can find the unique colors we have for example

In [ ]:
# Since we don't have any duplicated colors, we'll concatenate a couple
tensorRGB2 = torch.cat((torch.tensor([[151,  70,  89], [78, 185, 149]]), tensorRGB))
tensorRGB2

tensor([[151,  70,  89],
        [ 78, 185, 149],
        [ 75,  62, 186],
        [ 89, 224, 249],
        [ 78, 185, 149],
        [151,  70,  89],
        [ 32, 198,  24]])

In [ ]:
# Here is important to use dim=0, because we want to compare the colors [r, g, b] over the first dimension, 
# otherwise we'll compare every element, which is not the case.

uniques, indices, counts = torch.unique(tensorRGB2, return_inverse=True, return_counts=True, dim=0)
print('We can see there are {} unique colors'.format(len(uniques)))
print(uniques)
print('\nInverse indices: \n{}'.format(indices))
print('\nCounts: \n{}'.format(counts))


We can see there are 5 unique colors
tensor([[ 32, 198,  24],
        [ 75,  62, 186],
        [ 78, 185, 149],
        [ 89, 224, 249],
        [151,  70,  89]])

Inverse indices: 
tensor([4, 2, 1, 3, 2, 4, 0])

Counts: 
tensor([1, 1, 2, 1, 2])


#### 5.3 [ Error: ValueError ] If return_inverse=False, the inverse_indices are not returned

In [ ]:
uniques, indices = torch.unique(torch.tensor([1, 2, 3, 3]))

ValueError: ignored

## **6. Serialization, tensor.save / tensor.load**

Saves an object to a disk file, E.g. a Tensor

#### 6.1 Basic example to save and load

Here we'll save a tensor as 'tensor.pt' on the drive. A common PyTorch convention is to save tensors using .pt file extension.

In [ ]:
# Saving the tensor
fileTensor = torch.tensor([0, 1, 2, 3, 4])
torch.save(fileTensor, 'tensor.pt')

In [ ]:
# Loading the tensor saved
loadedTensor = torch.load('tensor.pt')
print(loadedTensor)

tensor([0, 1, 2, 3, 4])


#### 6.3 [ Error: FileNotFoundError ] The object does not exist or is not found

In [ ]:
torch.load('noFile.pt')

FileNotFoundError: ignored

## Conclusion

Now that we have covered some useful functions to create tensors and perform mathematical operations with them, should not be so hard to keep analyzing some other similar functions. PyTorch offers many and it's worth it to look deeper.


Next step would be to start analyzing **torch.nn** so we can move on with the main purpose of PyTorch, which is to provide libraries for Machine and Deep Learning.

## Reference Links
Provide links to your references and other interesting articles about tensors
* Official documentation for tensor operations: https://pytorch.org/docs/stable/torch.html
* PyTorch Autograd: https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95
* Matrix transpose: https://simple.wikipedia.org/wiki/Transpose#Examples
* Use of torch.abs on Tensors: https://pennylane.ai/qml/demos/pytorch_noise.html
* Broadcasting explained - Tensors for deep learning and neural networks: https://deeplizard.com/learn/video/6_33ulFDuCg

In [ ]:
jovian.commit(project='01-tensor-operations', files=['MOCK_DATA.csv'])

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Uploading additional files...
[jovian] Committed successfully! https://jovian.ai/hada-garcia/01-tensor-operations


'https://jovian.ai/hada-garcia/01-tensor-operations'